<a href="https://colab.research.google.com/github/moukouel/Notebooks/blob/main/PyWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch as th
import torch.nn as nn
import re
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.nn import Module, Sequential
import requests

from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim


print(th.__version__)



2.6.0+cu124


In [ ]:
# Check if GPU is available
print(th.cuda.is_available())
# Download the file
url = 'https://www.gutenberg.org/cache/epub/100/pg100.txt'
response = requests.get(url)

# Read and decode the text
text = response.content.decode(encoding='utf-8')

# Check the type and print the first 50 characters
print(type(text))  # <class 'str'>
print(text[:50])

# Get the length of the text
print(len(text))

False
<class 'str'>
﻿The Project Gutenberg eBook of The Complete Works
5575053


In [ ]:
#Preprocess the text
def preprocess_text(text):
    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    return text
# Preprocess the text
text = preprocess_text(text)
# Check the first 50 characters after preprocessing
print(text[:50])
# Get the length of the preprocessed text
print(len(text))

 the project gutenberg ebook of the complete works
5306848


In [ ]:

# Tokenization and vocabulary building
nltk.download('punkt')
nltk.download('punkt_tab')
tokens = word_tokenize(text)  # Tokenize the text into words

# Check the first 10 tokens
print("First 10 tokens:", tokens[:10])

# Check the number of tokens
print("Total number of tokens:", len(tokens))

# Count the number of unique tokens
unique_tokens = list(set(tokens))  # Create a list of unique tokens
print("Number of unique tokens:", len(unique_tokens))

# Construct the vocabulary
# Map each unique token to an index
vocab = {token: idx for idx, token in enumerate(unique_tokens)}

# Convert text to sequences of integers
sequences = [vocab[token] for token in tokens]
#Convert sequences into words
reverse_vocab = {idx: token for token, idx in vocab.items()}
# Convert sequences back to words
decoded_sequences = [reverse_vocab[idx] for idx in sequences]

# Vocabulary and sequence details
print("Vocabulary size:", len(vocab))
print("First 10 sequences:", sequences[:10])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


First 10 tokens: ['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'complete', 'works', 'of', 'william']
Total number of tokens: 1214698
Number of unique tokens: 29377
Vocabulary size: 29377
First 10 sequences: [15985, 20744, 18673, 12497, 25576, 15985, 14172, 8693, 25576, 15069]


In [ ]:

from sklearn.model_selection import train_test_split

# Create input-output pairs for training (next-word prediction task)
X, y = [], []
max_len = 3  # Define a fixed sequence length
for i in range(max_len, len(sequences)):
    X.append(sequences[i-max_len:i])  # Input: previous `max_len` words
    y.append(sequences[i])           # Output: next word (target)

# Convert to PyTorch tensors
X = th.tensor(X, dtype=th.long)
y = th.tensor(y, dtype=th.long)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Print dataset shapes
print("Training Set:", X_train.shape, y_train.shape)
print("Validation Set:", X_val.shape, y_val.shape)

Training Set: torch.Size([850286, 3]) torch.Size([850286])
Validation Set: torch.Size([364409, 3]) torch.Size([364409])


In [ ]:

# ==========================================
# Step 3: Build the RNN Model with GRU
# ==========================================
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, gru_units):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)  # Embedding layer

        # GRU layer with input size `embedding_dim`, hidden size `gru_units`
        # and `batch_first=True` to ensure input shape is (batch_size, seq_len, input_size)
        self.gru = nn.GRU(embedding_dim, gru_units, batch_first=True)  # GRU layer
        self.fc = nn.Linear(gru_units, vocab_size)  # Fully connected layer

    def forward(self, x):
        x = self.embedding(x)  # Apply embedding
        _, hidden = self.gru(x)  # GRU forward pass (we only need the hidden state)
        output = self.fc(hidden.squeeze(0))  # Apply dense layer
        return th.softmax(output, dim=-1)  # Apply softmax activation

# Model parameters
vocab_size = len(vocab)  # Vocabulary size
embedding_dim = 128
gru_units = 128

# Instantiate the model
model = GRUModel(vocab_size, embedding_dim, gru_units)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Display model architecture
print(model)

# Save model path
model_path = './wordgen_rnn_gru_model.pth'

GRUModel(
  (embedding): Embedding(29377, 128)
  (gru): GRU(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=29377, bias=True)
)


In [ ]:

# Create DataLoader for training and validation sets
batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# Create DataLoader for validation set
val_loader = DataLoader(val_dataset, batch_size=batch_size)



In [ ]:
# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs, model_path):
    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        # Initialize loss and accuracy
        epoch_loss = 0.0
        correct = 0
        total = 0

        # Training phase
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Clear gradients
            output = model(X_batch)  # Forward pass
            loss = criterion(output, y_batch)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            epoch_loss += loss.item()
            _, predicted = th.max(output, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

        train_accuracy = correct / total
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.4f}")

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with th.no_grad():
            for X_batch, y_batch in val_loader:
                output = model(X_batch)
                loss = criterion(output, y_batch)
                val_loss += loss.item()
                _, predicted = th.max(output, 1)
                val_correct += (predicted == y_batch).sum().item()
                val_total += y_batch.size(0)

        val_accuracy = val_correct / val_total
        print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}")

        model.train()  # Switch back to training mode

    # Save the trained model
    th.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=3, model_path=model_path)

Epoch 1/3
Epoch 1/3, Loss: 10.1725, Accuracy: 0.1159
Validation Loss: 10.1691, Validation Accuracy: 0.1191
Epoch 2/3


In [ ]:
# ==========================================
# Step 5: Load the Saved Model
# ==========================================
# Load the saved model
loaded_model = GRUModel(vocab_size, embedding_dim, gru_units)
loaded_model.load_state_dict(th.load(model_path))
loaded_model.eval()  # Set the model to evaluation mode

# Verify the model was loaded correctly
print(loaded_model)

GRUModel(
  (embedding): Embedding(29377, 128)
  (gru): GRU(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=29377, bias=True)
)


In [ ]:
import torch
import numpy as np
from tqdm import tqdm

def generate_text(model, vocab, seed_text, max_len, num_words=50, temperature=0.8):
    """
    Generate text using the trained PyTorch model with a temperature parameter.
    :param model: Trained PyTorch model.
    :param vocab: Vocabulary object for token-to-index and index-to-token mapping.
    :param seed_text: Initial input text to start generation.
    :param max_len: Maximum sequence length for padding.
    :param num_words: Number of words to generate.
    :param temperature: Temperature value for controlling randomness.
    :return: Generated text string.
    """
    model.eval()  # Set the model to evaluation mode
    # Check if the seed text is empty
    if not seed_text:
        raise ValueError("Seed text cannot be empty.")
    # Check if the seed text contains only valid tokens
    # Tokenize the seed text
    # Convert the seed text to lowercase and split into words
    seed_text = preprocess_text(seed_text)

    # Convert the seed text to a list of tokens
    tokens = [vocab[token] for token in seed_text.split() if token in vocab]  # Tokenize seed text

    #Convert sequences into words
    reverse_vocab = {idx: token for token, idx in vocab.items()}
    for _ in tqdm(range(num_words), desc="Generating text"):
        # Pad the sequence to the maximum length
        if len(tokens) < max_len:
            padded_tokens = [0] * (max_len - len(tokens)) + tokens  # Left-pad with zeros
        else:
            padded_tokens = tokens[-max_len:]  # Truncate to the last `max_len` tokens

        # Convert to tensor
        input_tensor = torch.tensor([padded_tokens], dtype=torch.long)

        # Predict the next word
        with torch.no_grad():
            predictions = model(input_tensor)  # Forward pass
            predictions = predictions[-1]  # Get predictions for the last time step
            predictions = predictions / temperature  # Apply temperature scaling
            probabilities = torch.softmax(predictions, dim=-1).numpy()  # Convert to probabilities
            probabilities = np.clip(probabilities, 1e-10, 1.0)  # Avoid zero probabilities

            # Normalize probabilities
            probabilities /= np.sum(probabilities)  # Ensure probabilities sum to 1
            # Sample the next word index
            predicted_word_index = np.random.choice(len(probabilities), p=probabilities)

            # Check if the predicted index is valid
            if predicted_word_index < 0 or predicted_word_index >= len(vocab):
                raise ValueError(f"Predicted word index {predicted_word_index} is out of bounds.")
            # Convert the predicted index to a word
            predicted_word = reverse_vocab[predicted_word_index]

            # Check if the predicted word is valid
            if predicted_word not in vocab:
                raise ValueError(f"Predicted word '{predicted_word}' is not in the vocabulary.")
            # Append the predicted word to the seed text
            tokens.append(predicted_word_index)
            seed_text += ' ' + predicted_word

    return seed_text

# Example of text generation
th.manual_seed(42)  # Set random seed for reproducibility
seed_text = "The project gutenberg ebook of"
print("Seed Text:", seed_text)
max_len = 10
generated_text = generate_text(model, vocab, seed_text, max_len, num_words=15, temperature=0.9)
#Save the generated text to a file
with open("generated_word.txt", "w") as f:
    f.write(generated_text)
# Print the generated text
print("Generated Text:", generated_text)

Seed Text: The project gutenberg ebook of


Generating text: 100%|██████████| 15/15 [00:00<00:00, 548.71it/s]

Generated Text: the project gutenberg ebook of death langley demonstrable gogs-wouns lubberly squash single inwardly neighs churlish sar senators commend keeper-back flush
